In [ ]:
import json
import time
import websocket
import ssl
import socks  # Provided by PySocks
import socket
from event import Event

def fetch_nostr_events(url, network, start, stop):
    # Generate a random subscription ID
    events = []
    sub_id = "" + str(int(time.time()))

    # Filter to get all events between start and stop
    req = [
        "REQ",
        sub_id,
        {
            "since": int(start),
            "until": int(stop)
        }
    ]

    close_req = ["CLOSE", sub_id]

    # Configure proxy if using Tor
    if network.lower() == "tor":
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
        socket.socket = socks.socksocket
        sslopt = {"cert_reqs": ssl.CERT_NONE}
    else:
        sslopt = {"cert_reqs": ssl.CERT_REQUIRED}

    def on_message(ws, message):
        print("Message received:", message)
        data = json.loads(message)
        if data[0] == "EVENT":
            try:
                Event.from_dict(data[2])
            except Exception as e:
                print(e, data[2])
            print("Event:", data[2])
            events.append(data[2])
        elif data[0] == "EOSE":
            print("End of stored events.")
            ws.send(json.dumps(close_req))
            ws.close()

    def on_error(ws, error):
        print("Error:", error)

    def on_close(ws, close_status_code, close_msg):
        print("Connection closed.")


    def on_open(ws):
        ws.send(json.dumps(req))

    print(f"Connecting to {url} over {network}...")
    ws = websocket.WebSocketApp(
        url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )

    ws.run_forever(sslopt=sslopt)
    print("Connection closed.")
    print(f"Fetched {len(events)} events.")
    return events

# Example usage:
events = fetch_nostr_events('wss://puravida.nostr.land', "clearnet", 1747296164-100, 1747296164)
events

Connecting to wss://puravida.nostr.land over clearnet...
Message received: ["AUTH","sD5AXRwcqPOHlQYysxc-qG"]
Message received: ["EOSE","1747572830"]
End of stored events.
Connection closed.
Connection closed.
Fetched 0 events.


[]